In [8]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),"..")))
from TextClassification import Augment
from TextClassification import TextPreprocess
from TextClassification import ExtractKeyword
from TextClassification import TextVector
from TextClassification import FeatureDimensionReduce
from TextClassification import FeatureCode
from TextClassification import ModelPreparation
from TextClassification import Classifier
from TextClassification import Successor
import pandas as pd 
import numpy as np
from sklearn import preprocessing
import json
from fastcore.transform import Pipeline

# 数据准备

In [9]:
data = pd.read_csv(os.getcwd()+'/data/训练集.csv')[:1000]
data['feature'] = data['评论标题'].map(str)+data['评论内容'].map(str)
data_feature = data['feature']

data_predict_origin = pd.read_csv(os.getcwd()+'/data/测试集.csv')[:1000]
data_predict_origin['feature'] = data_predict_origin['评论标题'].map(str)+data_predict_origin['评论内容'].map(str)
data_predict_feature = data_predict_origin['feature']

In [10]:
# 加载label_array
data['label'] = preprocessing.LabelEncoder().fit_transform(data['评分'])
label_array = np.array(data['label'])
# label与类别的映射关系
label_map = dict(zip(data['评分'].unique(),preprocessing.LabelEncoder().fit_transform(data['评分'].unique())))
print('label对应关系:',label_map)

label对应关系: {4.0: 3, 5.0: 4, 1.0: 0, 3.0: 2, 2.0: 1}


In [11]:
stopwords_cy = list(pd.read_csv(os.getcwd()+'/data/define_stopwords.txt',names = ['word'], 
                                sep = 'aaa',encoding = 'UTF-8',engine='python').word)

In [12]:
parameter_file = open(os.getcwd()+'/data/parameter.json', "rb")
parameter_json = json.load(parameter_file) 

In [13]:
# 构建input_data
input_data = {}
input_data['data_feature'] = data_feature
input_data['stop_words'] = stopwords_cy
input_data['label_array'] = label_array
input_data['aug_mark'] = np.array([0 for i in range(len(label_array))])
input_data['parameter'] = parameter_json
input_data['data_predict_feature'] = data_predict_feature

# pipeline

In [15]:
pip = Pipeline([TextPreprocess.typeface_transform.char_converter,
                TextPreprocess.cut_word.jieba_cut,
                TextPreprocess.chartype_filter.number_filter,
                TextVector.traditional_vector.one_hot_keras,
                ModelPreparation.dataset_partition.split_train_test,
                Classifier.deep_learning.train.text_inception])
pip_output = pip(input_data.copy())

2022-07-17 18:37:07,105 : INFO : 文体转换已完成
100%|██████████| 1000/1000 [00:00<00:00, 565651.25it/s]
2022-07-17 18:37:09,030 : INFO : jieba分词已完成
100%|██████████| 1000/1000 [00:00<00:00, 45922.70it/s]
2022-07-17 18:37:09,076 : INFO : 特定字符过滤已完成
2022-07-17 18:37:09,126 : INFO : onehot已完成
2022-07-17 18:37:13,164 : INFO : 数据拆分已完成


Epoch 1/10
19/19 [==============================] - 8s 357ms/step - loss: 1.6034 - sparse_categorical_accuracy: 0.3919 - val_loss: 1.6709 - val_sparse_categorical_accuracy: 0.5050
Epoch 2/10
19/19 [==============================] - 5s 253ms/step - loss: 1.1342 - sparse_categorical_accuracy: 0.6070 - val_loss: 2.1702 - val_sparse_categorical_accuracy: 0.5050
Epoch 3/10
19/19 [==============================] - 5s 268ms/step - loss: 0.5497 - sparse_categorical_accuracy: 0.8219 - val_loss: 1.4067 - val_sparse_categorical_accuracy: 0.5050
Epoch 4/10
19/19 [==============================] - 5s 269ms/step - loss: 0.1982 - sparse_categorical_accuracy: 0.9555 - val_loss: 1.4489 - val_sparse_categorical_accuracy: 0.5050
Epoch 5/10
19/19 [==============================] - 5s 273ms/step - loss: 0.0948 - sparse_categorical_accuracy: 0.9889 - val_loss: 1.4274 - val_sparse_categorical_accuracy: 0.5050
Epoch 6/10
19/19 [==============================] - 5s 270ms/step - loss: 0.0574 - sparse_categorica

In [31]:
pip_outputs = [pip_output]
evaluation = Successor.ModelEvaluation(f1_score='micro')
evaluation_result = evaluation.run(pip_outputs)

100%|██████████| 1/1 [00:07<00:00,  7.15s/it]


In [32]:
prediction = Successor.ModelPredict(return_type='class')
prediction_result = prediction.run(pip_output['clf_model'], pip_output['docvec_predict_array'])

In [20]:
save = Successor.ModelSave(save_path = os.getcwd()+'/save',name = 'rcnn')
save.run(pip_output['clf_model'])

pkl模型保存方案失败,已切换为keras模型保存方案


2022-01-24 00:24:51,200 : INFO : Assets written to: /Users/niejikai/Desktop/垃圾文件//rcnn/assets


模型已保存在/Users/niejikai/Desktop/垃圾文件//rcnn/
